## 영상불러와서 OD

In [ ]:
import cv2
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

def detect_objects_on_video(video_path, model, threshold=0.5):
    # 비디오를 열고 비디오의 프레임을 처리합니다.
    cap = cv2.VideoCapture(video_path)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                if prediction[0]['scores'][i] > threshold:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 객체 검출 실행
detect_objects_on_video('path_to_your_video.mp4', model)


## 내장카메라

In [11]:
import cv2
import torch
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

def detect_objects_on_webcam(model, threshold=0.5):
    # 웹캠으로부터 비디오를 캡처합니다.
    cap = cv2.VideoCapture(0)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                if prediction[0]['scores'][i] > threshold:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 웹캠에서 객체 검출을 실행
detect_objects_on_webcam(model)
